<div style="
  padding: 30px;
  text-align: center;" class='row'>
<div style="float:left;width: 15%;" class='column'><a href="https://www.colombiacompra.gov.co"><img alt="Logo Colombia Compra Eficiente" id="logocce" src="https://www.colombiacompra.gov.co/sites/cce_public/files/Images/logocce.png" style="height: 45px;"></a></div>
    <div style="float:left;width: 70%;" class='column'>
        <h1> Validación entre las bases de Tendery, DataVault y DataLake
        </h1> 
    </div>
<div style="float:left;width: 15%;" class='column'><a href="https://www.dnp.gov.co/" target="_blank"><img class="float-right" id="logodnp" src="https://www.dnp.gov.co/img/logoNuevo.jpg" style="width: 200px;"></a></div>
</div>

El presente cuadernillo tiene como objetivo llevar a cabo la validación de las fuentes de datos utilizadas en el proyecto de bases unificadas. Nos enfocaremos en las fuentes de datos "Tendery", "Datavault" y "Datalake", con el propósito de garantizar la integridad y consistencia de la información contenida en ellas.

La validación de las fuentes de datos es un paso crítico en cualquier proyecto de análisis de datos, ya que nos permite asegurarnos de que la información utilizada para tomar decisiones y realizar análisis sea confiable y precisa. La consistencia y calidad de los datos son factores clave para obtener resultados precisos y confiables.

In [1]:
# =============================================================================================
# Librerías
# =============================================================================================

from libreria import login # Configuración de credenciales
import pandas as pd # Manejo de datos

# =============================================================================================
# Configuración de credenciales
# =============================================================================================

cnxn_reports, cursor_reports = login.login_sql(login.tipo_bodega.REPORTS) # Reports (Tendery)
cnxn_sandbox, cursor_sandbox = login.login_sql(login.tipo_bodega.SANDBOX) # Sandbox (DataVault)
# blob_service_client, container_client = login.login_datalake() # DataLake

## Validación de bases de contratos

En esta sección, nos enfocaremos en la validación de las bases de contratos provenientes de las fuentes de datos previamente mencionadas. A continuación, describiremos los pasos a seguir para llevar a cabo la validación de las bases de contratos:

### Carga de las bases de contratos

In [2]:
# ============================================================================================================================
# DataVault
# ============================================================================================================================
dv_cantidad_contratos = pd.read_sql_query("SELECT COUNT (*) FROM [DMDataVault].[Dbo_FactProcurementContract]", cnxn_sandbox)
dv_contratos = pd.read_sql_query("SELECT TOP 5 * FROM [DMDataVault].[Dbo_FactProcurementContract]", cnxn_sandbox)
dv_conteo_y_actualizacion_por_fecha_y_id_contrato = pd.read_sql_query(
    """
    SELECT DISTINCT
    YEAR(LoadDate) AS 'AÑO',
    MONTH (LoadDate) AS 'MES',
    DAY (LoadDate) AS 'DIA',
    COUNT (DISTINCT ProcurementContractId) AS 'ID CONTRATO'
    FROM [DMDataVault].[Dbo_FactProcurementContract]
    GROUP BY YEAR(LoadDate),MONTH (LoadDate),DAY (LoadDate)
    ORDER BY YEAR(LoadDate)desc,MONTH (LoadDate)desc,DAY (LoadDate)desc
    """
    , cnxn_sandbox)

# ============================================================================================================================
# Tendery
# ============================================================================================================================
t_cantidad_contratos = pd.read_sql_query("SELECT COUNT (*) FROM [DM-CCE-Tendering].[dbo].[FactProcurementContract]", cnxn_reports)
t_contratos = pd.read_sql_query("SELECT top 5 * FROM [DM-CCE-Tendering].[dbo].[FactProcurementContract]", cnxn_reports)
t_conteo_y_actualizacion_por_fecha_y_id_contrato = pd.read_sql_query(
    """
    SELECT DISTINCT
    YEAR(ApprovalDate) AS 'AÑO',
    MONTH (ApprovalDate) AS 'MES',
    DAY (ApprovalDate) AS 'DIA',
    COUNT ( DISTINCT UniqueIdentifier) AS 'ID PROCESO'
    FROM [DM-CCE-Tendering].[dbo].[FactProcurementContract]
    GROUP BY YEAR(ApprovalDate),MONTH (ApprovalDate),DAY (ApprovalDate)
    ORDER BY YEAR(ApprovalDate)desc,MONTH (ApprovalDate)desc,DAY (ApprovalDate)desc
    """
    , cnxn_reports)

# ============================================================================================================================
# DataLake
# ============================================================================================================================
dl_contratos = pd.read_parquet("../../../muestras de datos/procesados/SECOP_LIMPIO_08_06.parquet")

C:\Users\Jorge\AppData\Local\Temp\ipykernel_24876\3289061625.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dv_cantidad_contratos = pd.read_sql_query("SELECT COUNT (*) FROM [DMDataVault].[Dbo_FactProcurementContract]", cnxn_sandbox)
C:\Users\Jorge\AppData\Local\Temp\ipykernel_24876\3289061625.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dv_contratos = pd.read_sql_query("SELECT TOP 5 * FROM [DMDataVault].[Dbo_FactProcurementContract]", cnxn_sandbox)
C:\Users\Jorge\AppData\Local\Temp\ipykernel_24876\3289061625.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are

### Análisis exploratorio utilizando `.info()`

In [4]:
dv_contratos.info() # DataVault

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 69 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   ProcurementContractId                    5 non-null      object        
 1   UniqueIdentifier                         5 non-null      object        
 2   FactAwardUniqueIdentifier                1 non-null      object        
 3   FactProcedureRequestUniqueIdentifier     5 non-null      object        
 4   FactProcedureReplySentUniqueIdentifier   1 non-null      object        
 5   Reference                                5 non-null      object        
 6   Description                              5 non-null      object        
 7   DimProcurementContractStateId            5 non-null      int64         
 8   DimProcurementContractAggregatedStateId  5 non-null      int64         
 9   RenewalDate                              0 non-

In [5]:
t_contratos.info() # Tendery

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 67 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   UniqueIdentifier                         5 non-null      object        
 1   AwardUniqueIdentifier                    5 non-null      object        
 2   FactProcedureRequestUniqueIdentifier     5 non-null      object        
 3   FactProcedureReplySentUniqueIdentifier   5 non-null      object        
 4   Reference                                5 non-null      object        
 5   Description                              4 non-null      object        
 6   DimProcurementContractStateId            5 non-null      int64         
 7   RenewalDate                              1 non-null      datetime64[ns]
 8   CategorizationCode                       5 non-null      object        
 9   CategorizationNorm                       5 non-

In [6]:
dl_contratos.info() # DataLake

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11915267 entries, 0 to 11915266
Data columns (total 30 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   index                        int64         
 1   Fuente                       object        
 2   ID Contrato                  object        
 3   ID Proceso                   object        
 4   ID entidad Plataforma        object        
 5   Entidad                      object        
 6   NIT Entidad                  object        
 7   Orden                        object        
 8   Modalidad                    object        
 9   Estado                       object        
 10  Descripción proceso          object        
 11  Objeto Contrato              object        
 12  Tipo de contrato             object        
 13  Fecha de Firma               datetime64[ns]
 14  UNSPSC                       object        
 15  Nombre Proveedor             object        
 16

### Análisis estadístico utilizando `.describe()`

In [8]:
dv_contratos.describe() # DataVault

,DimProcurementContractStateId,DimProcurementContractAggregatedStateId,ContractValue,ContractStartDate,ContractEndDate,SendToSupplierDate,ApprovalDate,NumberOfAdvancedPriceListItems,TypeId,IsProcurementContractStandardOrder,LiquidationStartDate,LiquidationEndDate,CreateDate,ExtendedDays,LoadDate,Duration,DurationType,ContractConfirmedDocument,ContractInExecutionDocument
count,5.0,5.0,5.000000e+00,5,5,5,5,5.0,5.0,5.0,3,3,5,5.0,5,5.000000,5.000000,5.000000e+00,5.000000e+00
mean,57.0,99.0,1.009075e+09,2021-12-09 17:24:00,2022-04-15 08:23:36,2021-12-03 09:29:57.534877696,2021-12-06 00:01:34.755114752,0.0,0.0,1.0,2022-05-15 15:40:00,2023-10-22 20:39:40,2021-12-02 00:28:39.378531840,0.0,2022-12-11 00:35:39.400000,98.000000,0.400000,4.416947e+08,4.494500e+08
min,57.0,99.0,4.200000e+06,2021-10-22 23:00:00,2021-10-29 23:00:00,2021-10-22 14:16:48.850155,2021-10-22 22:00:39.740186,0.0,0.0,1.0,2021-12-23 13:00:00,2022-11-23 04:59:00,2021-10-21 14:25:38.104336,0.0,2022-10-03 10:08:06,7.000000,0.000000,4.170385e+08,4.222064e+08
25%,57.0,99.0,1.279333e+07,2021-12-09 17:00:00,2021-12-23 04:00:00,2021-12-08 20:38:24.362074880,2021-12-09 15:08:37.195472896,0.0,0.0,1.0,2022-04-06 21:00:00,2023-03-22 16:59:30,2021-12-08 19:57:51.412793088,0.0,2022-10-03 10:08:06,8.000000,0.000000,4.436505e+08,4.436508e+08
50%,57.0,99.0,2.014965e+07,2021-12-13 13:00:00,2022-07-01 04:59:00,2021-12-10 19:15:08.123951104,2021-12-10 20:17:46.431311872,0.0,0.0,1.0,2022-07-20 05:00:00,2023-07-20 05:00:00,2021-12-08 19:59:00.655179008,0.0,2022-10-03 10:08:06,10.000000,0.000000,4.448001e+08,4.453566e+08
75%,57.0,99.0,3.295920e+07,2021-12-17 17:00:00,2022-07-20 05:00:00,2021-12-10 23:49:29.987156992,2021-12-16 21:21:04.841888,0.0,0.0,1.0,2022-07-26 05:00:00,2024-04-06 16:30:00,2021-12-08 21:08:54.186170112,0.0,2023-01-26 08:46:17,210.000000,0.000000,4.481834e+08,4.501786e+08
max,57.0,99.0,4.975273e+09,2022-01-14 17:00:00,2022-08-01 04:59:00,2021-12-24 17:29:56.351050,2021-12-30 17:19:45.566715,0.0,0.0,1.0,2022-08-01 05:00:00,2024-12-24 04:00:00,2021-12-22 22:51:52.534182,0.0,2023-05-19 11:47:42,255.000000,2.000000,4.548012e+08,4.858575e+08
std,0.0,0.0,2.217197e+09,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,123.812358,0.894427,1.445062e+07,2.301434e+07


In [9]:
t_contratos.describe() # Tendery

,DimProcurementContractStateId,RenewalDate,ContractValue,DeliverStartDate,DeliverEndDate,ContractStartDate,ContractEndDate,SendToSupplierDate,ApprovalDate,NumberOfAdvancedPriceListItems,TypeId,LiquidationStartDate,LiquidationEndDate,CreateDate,AdtInsertedAt,AdtEditedAt,DimProcurementContractAggregatedStateId,ExtendedDays
count,5.000000,1,5.000000e+00,4,4,4,4,4,4,5.0,5.0,1,1,5,5,5,5.000000,1.0
mean,18.000000,2019-11-18 22:00:00,2.418954e+07,2017-08-16 09:15:00,2018-07-02 14:30:00,2017-08-16 09:15:00,2018-07-02 14:30:00,2017-11-20 10:06:39.718499840,2018-01-13 01:38:24.842500096,0.0,0.0,2019-06-18 22:00:00,2021-01-03 22:00:00,2017-12-27 15:56:40.548800,2018-01-01 19:48:48,2019-11-03 19:47:12,97.000000,0.0
min,0.000000,2019-11-18 22:00:00,1.200000e+06,2017-01-04 13:00:00,2018-01-01 04:00:00,2017-01-04 13:00:00,2018-01-01 04:00:00,2017-02-22 15:01:17.097000,2017-04-26 20:41:53.507000,0.0,0.0,2019-06-18 22:00:00,2021-01-03 22:00:00,2017-01-05 00:32:18.440000,2017-01-13 08:18:00,2019-06-18 01:59:00,96.000000,0.0
25%,10.000000,2019-11-18 22:00:00,1.789792e+07,2017-01-04 13:00:00,2018-01-01 04:00:00,2017-01-04 13:00:00,2018-01-01 04:00:00,2017-03-20 22:42:06.644250112,2017-07-26 10:19:40.266499840,0.0,0.0,2019-06-18 22:00:00,2021-01-03 22:00:00,2017-01-05 00:35:14.940000,2017-01-13 08:18:00,2019-08-01 02:02:00,96.000000,0.0
50%,10.000000,2019-11-18 22:00:00,2.774500e+07,2017-01-04 13:00:00,2018-01-01 04:00:00,2017-01-04 13:00:00,2018-01-01 04:00:00,2017-06-20 15:48:57.788499968,2017-09-04 04:08:39.935000064,0.0,0.0,2019-06-18 22:00:00,2021-01-03 22:00:00,2017-01-05 01:31:13.430000128,2017-01-13 08:18:00,2019-08-01 02:02:00,96.000000,0.0
75%,17.000000,2019-11-18 22:00:00,2.856840e+07,2017-08-16 09:15:00,2018-07-02 14:30:00,2017-08-16 09:15:00,2018-07-02 14:30:00,2018-02-20 03:13:30.862749952,2018-02-21 19:27:24.511000064,0.0,0.0,2019-06-18 22:00:00,2021-01-03 22:00:00,2019-06-15 11:45:58.527000064,2019-06-16 01:05:00,2019-08-01 02:02:00,98.000000,0.0
max,53.000000,2019-11-18 22:00:00,4.553640e+07,2019-06-18 22:00:00,2019-12-31 22:00:00,2019-06-18 22:00:00,2019-12-31 22:00:00,2019-06-19 17:47:26.200000,2019-06-20 01:34:25.993000,0.0,0.0,2019-06-18 22:00:00,2021-01-03 22:00:00,2019-06-15 17:18:37.407000,2019-06-16 01:05:00,2020-12-30 18:51:00,99.000000,0.0
std,20.481699,NaN,1.624506e+07,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.414214,NaN


In [7]:
dl_contratos.describe() # DataLake

,index,Fecha de Firma,Valor del contrato,Inicio de contrato,Fin de contrato
count,1.191527e+07,2483423,1.191527e+07,2490608,11871240
mean,4.181748e+06,2021-05-08 16:52:52.856307968,-1.303452e+12,2021-06-11 17:56:56.792177408,2020-01-27 20:47:28.746890240
min,0.000000e+00,2002-06-22 22:00:00,-9.223372e+18,2002-06-22 22:00:00,1899-12-30 00:00:00
25%,1.500806e+06,2020-07-23 23:59:00,4.400000e+06,2020-08-21 08:00:00,2017-12-28 00:00:00
50%,3.560388e+06,2021-10-22 10:23:00,1.100000e+07,2021-11-01 12:00:00,2020-05-06 00:00:00
75%,6.826630e+06,2022-09-19 12:00:00,2.548438e+07,2022-09-20 15:00:15,2022-01-01 00:00:00
max,1.002933e+07,2030-12-01 12:00:00,6.004813e+18,2030-12-01 12:00:00,2222-09-25 23:59:00
std,3.027508e+06,NaN,7.942238e+15,NaN,NaN


# Procesos

In [3]:
# ============================================================================================================================
# DataVault
# ============================================================================================================================
dm_cantidad_procesos = pd.read_sql_query("SELECT COUNT (*) FROM [DMDataVault].[Dbo_FactProcedureRequest]", cnxn_sandbox)
dm_procesos = pd.read_sql_query("SELECT TOP 5 * FROM [DMDataVault].[Dbo_FactProcedureRequest]", cnxn_sandbox)
dm_conteo_y_actualizacion_por_fecha_y_id_proceso = pd.read_sql_query(
    """
    SELECT DISTINCT
    YEAR(LoadDate) AS 'AÑO',
    MONTH (LoadDate) AS 'MES',
    DAY (LoadDate) AS 'DIA',
    COUNT (DISTINCT ProcedureRequestId) AS 'ID PROCESO'
    FROM [DMDataVault].[Dbo_FactProcedureRequest]
    GROUP BY YEAR(LoadDate),MONTH (LoadDate),DAY (LoadDate)
    ORDER BY YEAR(LoadDate)desc,MONTH (LoadDate)desc,DAY (LoadDate)desc
    """
    , cnxn_sandbox)

# ============================================================================================================================
# Tendery
# ============================================================================================================================
t_cantidad_procesos = pd.read_sql_query("SELECT COUNT (*) FROM [DM-CCE-Tendering].[dbo].[FactProcedureRequest]", cnxn_reports)
t_procesos = pd.read_sql_query("SELECT top 5 * FROM [DM-CCE-Tendering].[dbo].[FactProcedureRequest]", cnxn_reports)
t_conteo_y_actualizacion_por_fecha_y_id_proceso = pd.read_sql_query(
    """
    SELECT DISTINCT
    YEAR(CreateDate) AS 'AÑO',
    MONTH (CreateDate) AS 'MES',
    DAY (CreateDate) AS 'DIA',
    COUNT ( DISTINCT UniqueIdentifier) AS 'ID PROCESO'
    FROM [DM-CCE-Tendering].[dbo].[FactProcedureRequest]
    GROUP BY YEAR(CreateDate),MONTH (CreateDate),DAY (CreateDate)
    ORDER BY YEAR(CreateDate)desc,MONTH (CreateDate)desc,DAY (CreateDate)desc
    """
    , cnxn_reports)

# ============================================================================================================================
# DataLake
# ============================================================================================================================
# dl_contratos = pd.read_parquet("../../../../Base unificada/SECOP_LIMPIO_08_06.parquet")

C:\Users\Jorge\AppData\Local\Temp\ipykernel_6376\646637958.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dm_cantidad_procesos = pd.read_sql_query(query_dm_cantidad_procesos, cnxn_sandbox)
C:\Users\Jorge\AppData\Local\Temp\ipykernel_6376\646637958.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dm_procesos = pd.read_sql_query(query_dm_procesos, cnxn_sandbox)
C:\Users\Jorge\AppData\Local\Temp\ipykernel_6376\646637958.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dm_conteo_y_actualizacion_por_fecha_y_id_proc